In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [24]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')

# Pick date
dates = feed.get_dates()


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [30]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [36]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

,date,trip_id,route_id,route_short_name,crossing_time,orientation
362,20140526,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:42:28,-1
363,20140526,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:50:28,1
68,20140527,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:42:28,-1
69,20140527,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:50:28,1
0,20140528,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:42:28,-1
1,20140528,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:50:28,1
262,20140529,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:42:28,-1
263,20140529,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:50:28,1
136,20140530,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:42:28,-1
137,20140530,CNS2014-CNS_MUL-Weekday-00-4166383,120-423,120,05:50:28,1


In [87]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

,num_trip_starts,num_trips,service_distance,service_duration,service_speed
2014-12-27 00:00:00,161.0,9.061111,3560.126691,108.733333,32.741815
2014-12-27 12:00:00,276.0,16.805556,6351.399405,201.666667,31.494543
2014-12-28 00:00:00,80.0,4.711111,1820.286021,56.533333,32.198456
2014-12-28 12:00:00,186.0,11.762500,4570.560294,141.150000,32.380873


In [67]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

CPU times: user 896 ms, sys: 0 ns, total: 896 ms
Wall time: 895 ms


indicator           num_trips                                            \
stop_id                750000 750001 750002 750003 750004 750005 750006   
2014-05-26 00:00:00        13     13     13     13     13     13     12   
2014-05-26 12:00:00        17     17     17     17     17     17     18   

indicator                                 ...                                \
stop_id             750007 750008 750009  ...   750437 750440 750448 750449   
2014-05-26 00:00:00     12     12     12  ...        6      5     10    110   
2014-05-26 12:00:00     18     18     18  ...       10     12     11    179   

indicator                                                      
stop_id             750450 750452 750453 750454 750455 750456  
2014-05-26 00:00:00     36     28     25     11      4     46  
2014-05-26 12:00:00     65     53     49     17     11     79  

[2 rows x 416 columns]

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()